In [ ]:
%matplotlib widget

In [ ]:
from genSTEM import ImageModel
import cupy as cp
import numpy as np
import matplotlib.pyplot as plt
from ase.io import read
from ase.build import make_supercell
atoms = read(r"C:\Users\thomasaar\Downloads\imeall-master\imeall\examples\boundary_interstitials\0013687130_v6bxv2_tv0.1bxv0.0_d1.8z_traj.xyz")
atoms = make_supercell(atoms, np.diag((1,2,1)))
mask = atoms.positions[:,2] > 35
del atoms[mask]
atoms.rotate([1,0,0], (0,0,1))

In [1]:
jitter = 3

m1 = ImageModel(atoms, 
                jitter_strength=jitter, 
                scan_rotation=0)
img1 = m1.generate_cupy()

m2 = ImageModel(atoms, 
                jitter_strength=jitter, 
                scan_rotation=90)
img2 = m2.generate_cupy()

img2 = cp.rot90(img2, 3)
side = np.minimum(*img1.shape)

img1 = img1[:side, :side]
img2 = img2[:side, :side]


NameError: name 'ImageModel' is not defined

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2)
ax1.imshow(img1.get())
ax2.imshow(img2.get())

In [83]:
hamm = np.hamming(img1.shape[0])
window = hamm[:, None] * hamm

def hybrid_correlation(img1, img2, p=0.9, filter=True):
    if filter:
        hamm = cp.hamming(img1.shape[0])
        window = hamm[:, None] * hamm
        img1 = img1 * window
        img2 = img2 * window

    fftimg1 = cp.fft.fft2(img1)
    fftimg2 = cp.fft.fft2(img2)
    m = fftimg1 * cp.conj(fftimg2)
    corr =  cp.real(cp.fft.ifft2(cp.abs(m)**p * cp.exp(1j * cp.angle(m))))
    corr = cp.fft.fftshift(corr)
    translation = cp.array(cp.unravel_index(corr.argmax(), corr.shape))
    center = cp.array(corr.shape) // 2
    return tuple(x for x in translation - center)

def translate(img, shift):
    return cp.roll(img, shift, axis=(0,1))

In [84]:
img2b = translate(img2, hybrid_correlation(img1, img2))
plt.figure()
plt.imshow((img1 + img2b).get())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [78]:
I2 = img1.get()
I2 = img2.get()

In [24]:
from scipy.ndimage import fourier_shift

In [56]:
I1, I2 = img1.get(), img2.get()
I3 = np.fft.ifft2(fourier_shift(np.fft.fft2(I2), S)).real
I4 = np.roll(I2, S[0], axis=0)
I4 = np.roll(I4, S[1], axis=1)
I4 = np.roll(I2, S, axis=(0,1))
I5 = scipy.ndimage.shift(I2, S)

plt.figure()
plt.imshow(I1 + I4)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [55]:
I4 = np.roll(I2, S, axis=(0,1))


In [29]:
plt.figure()
plt.imshow(I2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
corr = scipy.signal.correlate2d(img1.get(), img2.get())

In [237]:
plt.figure()
plt.imshow(window)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
window = 

In [13]:
data = np.random.random((300, 300))
%timeit np.real(np.fft.ifft(data))
%timeit np.fft.irfft(data)

957 µs ± 22.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
2.92 ms ± 84.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
from py4DSTEM.process.utils import get_maxima_2D

In [ ]:
get_maxima_2D()

In [19]:
# a = np.real(np.fft.ifft(data))
# b = np.fft.irfft(data)
# np.testing.assert_array_almost_equal(a, b)

In [15]:
a.shape

(300, 300)

In [16]:
b.shape

(300, 598)

In [5]:
import matplotlib.pyplot as plt
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10,10))
ax1.imshow(cp.asnumpy(img1))
ax2.imshow(cp.asnumpy(cp.rot90(img2, 3)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
m.generate_cupy(jitt)